In [1]:
import os
import pickle
import json
import numpy as np
from keras.layers import Input,Dense
from keras.models import Model
from keras.applications.nasnet import NASNetMobile
from keras.applications.nasnet import preprocess_input
from keras.applications.nasnet import decode_predictions

from keras.layers import AveragePooling2D
from keras.preprocessing import image
from keras.models import Model
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
K.set_session(sess)
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import f1_score

/home/parmeet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/parmeet/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
num_labels = 228 + 1

In [3]:
#getting annotations from file
with open('train.json') as train_json:
    train_data = json.load(train_json)
label_mapping = {}
for obj in train_data['annotations']:
    label_mapping[obj['imageId']] = obj['labelId']

In [4]:
#getting annotations from validation file
with open('validation.json') as val_json:
    validation_data = json.load(val_json)
validation_label_mapping = {}
for obj in validation_data['annotations']:
    validation_label_mapping[obj['imageId']] = obj['labelId']

In [5]:
#getting annotations from file
with open('test.json') as test_json:
    test_data = json.load(test_json)
len(test_data['images'])

39706

In [6]:
base_model = NASNetMobile(
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=True
)

model = Model(
    inputs=base_model.input,
    outputs=base_model.get_layer(index=749).output
)

In [7]:
def extract_image(image_path,model):
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        return model.predict(x)

In [8]:
def extract_features_test(folder):
    if os.path.isfile(folder+"_data_nasnet.pckl"):
        with open(folder+"_data_nasnet.pckl",'rb') as write_file:
            image_features,img_ids = pickle.load(write_file)
    else:
        with open(folder+"_data_nasnet.pckl",'wb') as output:
            with open('test.json') as test_json:
                test_data = json.load(test_json)
            num_test_images = len(test_data['images'])

            image_features = []
            img_ids = []
            for i in range(num_test_images):
                print(i)
                try:
                    features = extract_image('data/test_images/'+str(i+1)+".jpg",model)
                except:
                    features = np.random.randn(1056)
                features = np.array(features).reshape(1,-1)
                image_features.append(features)
                img_ids.append(i+1)
            pickle.dump((image_features,img_ids),output)
    return image_features,img_ids

In [9]:
def extract_features(folder,validation_label_mapping):
    if os.path.isfile(folder+"_data_nasnet.pckl"):
        with open(folder+"_data_nasnet.pckl",'rb') as write_file:
            validation_features,val_labels = pickle.load(write_file)
    else:
        with open(folder+"_data_nasnet.pckl",'wb') as output:
            validation_files = os.listdir('data/'+folder+'_images/')
            num_validation_files = len(validation_files)
            batch_sz = 100
            num_batches = int(num_validation_files/batch_sz)

            print("Number of batches",num_batches)
            validation_features = []
            validation_labels = []
            for idx,file in enumerate(validation_files):
                print(idx)
                try:
                    image_feature = extract_image('data/'+folder+'_images/'+file,model)
                    validation_features.append(image_feature)
                    validation_labels.append(validation_label_mapping[file.split(".")[0]])
                except Exception as ex:
                    print("exception catching",ex)
            val_labels = np.zeros((len(validation_labels),num_labels))
            for idx,labs in enumerate(validation_labels):
                val_labels[idx][list(map(int, labs))] = 1        
            pickle.dump((validation_features,val_labels),output)
    return validation_features,val_labels

In [10]:
validation_features,validation_labels = extract_features('validation',validation_label_mapping)
test_features,test_ids = extract_features_test('test')

In [11]:
train_features,train_labels = extract_features('train',label_mapping)

In [ ]:
train_features.shape

In [12]:
image_input = Input(shape=(1056,))
middle = BatchNormalization()(image_input)
middle = Dense(2048,activation='sigmoid')(middle)
middle = BatchNormalization()(image_input)
middle = Dense(2048,activation='sigmoid')(image_input)
middle = BatchNormalization()(middle)
output = Dense(num_labels,activation='sigmoid')(middle)
model_dense = Model(image_input, output)
model_dense.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [13]:
train_files = os.listdir('data/train_images/')
num_train_files = len(train_files)
batch_sz = 100
num_batches = int(num_train_files/batch_sz)

scores = []
print("Number of batches",num_batches)
epochs = 3
for epoch in range(epochs):
    for i in range(num_batches):
        print("Batch number",i)
        batch_features = np.squeeze(np.array(train_features),axis=1)[i*batch_sz:i*batch_sz+batch_sz,:]
        batch_labels = np.array(train_labels)[i*batch_sz:i*batch_sz+batch_sz,:]
        model_dense.fit(batch_features,batch_labels,verbose=0)
        if i%10==0:
            predicted = model_dense.predict(np.squeeze(np.array(validation_features),axis=1))
            predicted[predicted>=0.3] = 1
            predicted[predicted<0.3] = 0
            f1 = f1_score(np.array(validation_labels),predicted,average='micro')
            print(f1)
            loss,accuracy_score = model_dense.evaluate(np.squeeze(np.array(validation_features),axis=1),np.array(validation_labels))
            scores.append((f1,loss,accuracy_score))
model_dense.save_weights('weights-nasnet')

Number of batches 739
Batch number 0
0.06806851434032532
9897/9897 [==============================] - 1s 108us/step
Batch number 1
Batch number 2
Batch number 3
Batch number 4
Batch number 5
Batch number 6
Batch number 7
Batch number 8
Batch number 9
Batch number 10
0.09415388337544027
9897/9897 [==============================] - 1s 61us/step
Batch number 11
Batch number 12
Batch number 13
Batch number 14
Batch number 15
Batch number 16
Batch number 17
Batch number 18
Batch number 19
Batch number 20
0.06873797646487798
9897/9897 [==============================] - 0s 50us/step
Batch number 21
Batch number 22
Batch number 23
Batch number 24
Batch number 25
Batch number 26
Batch number 27
Batch number 28
Batch number 29
Batch number 30
0.0821043857896017
9897/9897 [==============================] - 1s 52us/step
Batch number 31
Batch number 32
Batch number 33
Batch number 34
Batch number 35
Batch number 36
Batch number 37
Batch number 38
Batch number 39
Batch number 40
0.24355096520349195


Batch number 333
Batch number 334
Batch number 335
Batch number 336
Batch number 337
Batch number 338
Batch number 339
Batch number 340
0.38615633935465266
9897/9897 [==============================] - 1s 51us/step
Batch number 341
Batch number 342
Batch number 343
Batch number 344
Batch number 345
Batch number 346
Batch number 347
Batch number 348
Batch number 349
Batch number 350
0.39964807038592276
9897/9897 [==============================] - 1s 53us/step
Batch number 351
Batch number 352
Batch number 353
Batch number 354
Batch number 355
Batch number 356
Batch number 357
Batch number 358
Batch number 359
Batch number 360
0.40607305864454096
9897/9897 [==============================] - 1s 51us/step
Batch number 361
Batch number 362
Batch number 363
Batch number 364
Batch number 365
Batch number 366
Batch number 367
Batch number 368
Batch number 369
Batch number 370
0.4262687500751807
9897/9897 [==============================] - 1s 66us/step
Batch number 371
Batch number 372
Batch num

Batch number 665
Batch number 666
Batch number 667
Batch number 668
Batch number 669
Batch number 670
0.40429577102923797
9897/9897 [==============================] - 1s 57us/step
Batch number 671
Batch number 672
Batch number 673
Batch number 674
Batch number 675
Batch number 676
Batch number 677
Batch number 678
Batch number 679
Batch number 680
0.40562869598998574
9897/9897 [==============================] - 0s 50us/step
Batch number 681
Batch number 682
Batch number 683
Batch number 684
Batch number 685
Batch number 686
Batch number 687
Batch number 688
Batch number 689
Batch number 690
0.41859205331713684
9897/9897 [==============================] - 1s 52us/step
Batch number 691
Batch number 692
Batch number 693
Batch number 694
Batch number 695
Batch number 696
Batch number 697
Batch number 698
Batch number 699
Batch number 700
0.3944427034367492
9897/9897 [==============================] - 0s 48us/step
Batch number 701
Batch number 702
Batch number 703
Batch number 704
Batch num

9897/9897 [==============================] - 1s 52us/step
Batch number 261
Batch number 262
Batch number 263
Batch number 264
Batch number 265
Batch number 266
Batch number 267
Batch number 268
Batch number 269
Batch number 270
0.4413577508241927
9897/9897 [==============================] - 1s 53us/step
Batch number 271
Batch number 272
Batch number 273
Batch number 274
Batch number 275
Batch number 276
Batch number 277
Batch number 278
Batch number 279
Batch number 280
0.4363563674385706
9897/9897 [==============================] - 0s 47us/step
Batch number 281
Batch number 282
Batch number 283
Batch number 284
Batch number 285
Batch number 286
Batch number 287
Batch number 288
Batch number 289
Batch number 290
0.41318403217139593
9897/9897 [==============================] - 0s 48us/step
Batch number 291
Batch number 292
Batch number 293
Batch number 294
Batch number 295
Batch number 296
Batch number 297
Batch number 298
Batch number 299
Batch number 300
0.4516505873859684
9897/9897 [

9897/9897 [==============================] - 1s 57us/step
Batch number 591
Batch number 592
Batch number 593
Batch number 594
Batch number 595
Batch number 596
Batch number 597
Batch number 598
Batch number 599
Batch number 600
0.43086617258154036
9897/9897 [==============================] - 1s 51us/step
Batch number 601
Batch number 602
Batch number 603
Batch number 604
Batch number 605
Batch number 606
Batch number 607
Batch number 608
Batch number 609
Batch number 610
0.4376729968469444
9897/9897 [==============================] - 1s 56us/step
Batch number 611
Batch number 612
Batch number 613
Batch number 614
Batch number 615
Batch number 616
Batch number 617
Batch number 618
Batch number 619
Batch number 620
0.4529364109464897
9897/9897 [==============================] - 1s 51us/step
Batch number 621
Batch number 622
Batch number 623
Batch number 624
Batch number 625
Batch number 626
Batch number 627
Batch number 628
Batch number 629
Batch number 630
0.453409068702489
9897/9897 [=

Batch number 188
Batch number 189
Batch number 190
0.44125452531575
9897/9897 [==============================] - 0s 49us/step
Batch number 191
Batch number 192
Batch number 193
Batch number 194
Batch number 195
Batch number 196
Batch number 197
Batch number 198
Batch number 199
Batch number 200
0.4341249042454542
9897/9897 [==============================] - 1s 51us/step
Batch number 201
Batch number 202
Batch number 203
Batch number 204
Batch number 205
Batch number 206
Batch number 207
Batch number 208
Batch number 209
Batch number 210
0.44531940229708444
9897/9897 [==============================] - 0s 49us/step
Batch number 211
Batch number 212
Batch number 213
Batch number 214
Batch number 215
Batch number 216
Batch number 217
Batch number 218
Batch number 219
Batch number 220
0.44988972209969125
9897/9897 [==============================] - 0s 49us/step
Batch number 221
Batch number 222
Batch number 223
Batch number 224
Batch number 225
Batch number 226
Batch number 227
Batch number

0.4410677553336539
9897/9897 [==============================] - 1s 52us/step
Batch number 521
Batch number 522
Batch number 523
Batch number 524
Batch number 525
Batch number 526
Batch number 527
Batch number 528
Batch number 529
Batch number 530
0.44941099085092384
9897/9897 [==============================] - 1s 60us/step
Batch number 531
Batch number 532
Batch number 533
Batch number 534
Batch number 535
Batch number 536
Batch number 537
Batch number 538
Batch number 539
Batch number 540
0.42875890825124174
9897/9897 [==============================] - 1s 52us/step
Batch number 541
Batch number 542
Batch number 543
Batch number 544
Batch number 545
Batch number 546
Batch number 547
Batch number 548
Batch number 549
Batch number 550
0.45425368747508466
9897/9897 [==============================] - 1s 66us/step
Batch number 551
Batch number 552
Batch number 553
Batch number 554
Batch number 555
Batch number 556
Batch number 557
Batch number 558
Batch number 559
Batch number 560
0.4534640

In [14]:
# model_dense.load_weights('weights-nasnet')

In [15]:
outputs = []
for test_image in test_features:
    output = model_dense.predict(test_image)
    #print(output)
    output[output>=0.3] = 1
    output[output<0.3] = 0    
    for i in range(output.shape[0]):
        arr = np.where(output[i]==1)[0]
        outputs.append(" ".join(map(str, arr.tolist())))    

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame()

In [18]:
df['image_id'] = test_ids
df['label_id'] = outputs

In [19]:
df.to_csv("predictions.csv",index=False)

In [20]:
train_files = os.listdir('data/train_images/')
len(train_files)

73908

In [21]:
validation_predictions = model_dense.predict(np.squeeze(np.array(validation_features),axis=1))
test_predictions = model_dense.predict(np.squeeze(np.array(test_features),axis=1))
with open("nasnet_results.pckl",'wb') as write_file:
    pickle.dump((validation_predictions,validation_labels,test_predictions),write_file)

In [22]:
with open("nasnet_results.pckl",'rb') as file:
    a = pickle.load(file)